### Integração Numérica - Regra de Simpson 3/8

Este notebook apresenta a implementação da Regra de Simpson 3/8 para cálculo de integrais definidas, incluindo o erro de aproximação simples e o erro generalizado, utilizando orientação a objetos.


In [52]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Classe `IntegralTresOitavosSimpson`

In [53]:
class IntegralTresOitavosSimpson(Integral):
    def __init__(self, limite_inf: int = None, limite_sup: int = None, h: float = None, quant_pontos: int = None, x: list[float] = None, func_integral=None, func_derivada_integral=None) -> None:
        super().__init__(limite_inf, limite_sup, h, quant_pontos, x, func_integral, func_derivada_integral)

    def erro_generalizado(self):
        if self.func_derivada_integral is None:
            return None
        derivada_4_grau_limite_inf = self.func_derivada_integral(self.limite_inf)
        derivada_4_grau_limite_sup = self.func_derivada_integral(self.limite_sup)
        if derivada_4_grau_limite_inf is None or derivada_4_grau_limite_sup is None:
            return None
        maior = self.saber_maior(derivada_4_grau_limite_inf, derivada_4_grau_limite_sup)
        parcela_1 = (self.h**4 / 80) * (self.limite_sup - self.limite_inf)
        return abs(maior * parcela_1)

    def erro_simples(self):
        if self.func_derivada_integral is None:
            return None
        derivada_4_grau_limite_inf = self.func_derivada_integral(self.limite_inf)
        derivada_4_grau_limite_sup = self.func_derivada_integral(self.limite_sup)
        if derivada_4_grau_limite_inf is None or derivada_4_grau_limite_sup is None:
            return None
        maior = self.saber_maior(derivada_4_grau_limite_inf, derivada_4_grau_limite_sup)
        parcela_1 = (3*math.pow(self.h, 5)) / 80
        return abs(maior * parcela_1)

    def integral(self):
        if self.func_integral is not None:
            self.y = [self.func_integral(val) for val in self.x]

        quant_x = len(self.x)
        if quant_x % 3 == 1:
            b = self.x.pop()
            penultimo = self.x[-1]

            primeiro = self.y.pop(0)
            ultimo = self.y.pop()
            cte = (3*self.h) / 8
            soma3x = 0
            soma2x = 0
            cont = 0
            for i in range(0, len(self.y)):
                if cont >= 2:
                    soma2x += (2 * self.y[i])
                    cont = 0
                else:
                    soma3x += (3 * self.y[i])
                    cont += 1

            y_aux = self.func_integral(penultimo)
            y_b = self.func_integral(b)
            int_1 = (self.h / 2) * (y_aux + y_b)
            return (cte * (primeiro + soma2x + soma3x + ultimo)) + int_1
        else:
            primeiro = self.y.pop(0)
            ultimo = self.y.pop()
            cte = (3*self.h) / 8
            soma3x = 0
            soma2x = 0
            cont = 0
            for i in range(0, len(self.y)):
                if cont >= 2:
                    soma2x += (2 * self.y[i])
                    cont = 0
                else:
                    soma3x += (3 * self.y[i])
                    cont += 1
            return cte * (primeiro + soma3x + soma2x + ultimo)

### Exemplo de uso

In [54]:
int_3_8_simpson_1_questao = IntegralTresOitavosSimpson(
        limite_inf=0, 
        limite_sup=1, 
        h=0.1,
        func_integral=lambda x: x * math.sqrt(x**2 + 1),
        func_derivada_integral=lambda x: (-15 * x) / ((math.pow(x, 2)+ 1) ** (7/2))
       )
print(f'Erro Geral = {int_3_8_simpson_1_questao.erro_generalizado():.6f}')
print(f'F(x) = {round(int_3_8_simpson_1_questao.integral(), 10)}')

Erro Geral = 0.000002
F(x) = 0.5768046824


In [55]:
pontos_x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
dummy_deriv = lambda x: 1.0  # ou algo que retorne sempre float

int_simpson_38_pontos = IntegralTresOitavosSimpson(
    x=pontos_x,
    func_integral=lambda x: x * math.sqrt(x**2 + 1),
    func_derivada_integral=dummy_deriv
)
print(f'F(x) por pontos = {round(int_simpson_38_pontos.integral(), 10)}')


TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'